# Accuracy-Efficiency Table

Calculate Compute

Normalize job times by compute used.

In [14]:
# Packages
import pandas as pd
import os
import glob
import dataframe_image as dfi # NOTE: YOU MUST HAVE GOOGLE CHROME INSTALLED FOR THIS TO WORK CORRECTLY

## Job History Data

In [15]:
# jobs and cleaned up names for them
jobs_and_clean_names = {
                'Fit_Logistic_Regression_Classifier_All_Data': 'Logistic Regression-All Features',
                'Fit_Logistic_Regression_Classifier_All_Features_PCA' : 'Logistic Regression-All Features PCA',
                'Fit_Logistic_Regression_Classifier_Individual_Features_PCA' : 'Logistic Regression-Individual Features PCA',
                'Fit_Logistic_Regression_Classifier_Top3_Features' : 'Logistic Regression-Top 3 Features',
                'Fit_Logistic_Regression_Classifier_VGG' : 'Logistic Regression-VGG',
                #'Fit_SVM_Classifier_All_Data': 'SVM-All Features',
                'Fit_SVM_Classifier_All_Features_PCA': 'SVM-All Features PCA',
                'Fit_SVM_Classifier_Individual_Features_PCA': 'SVM-Individual Features PCA',
                'Fit_SVM_Classifier_Top3_Features': 'SVM-Top 3 Features',
                'Fit_SVM_Classifier_VGG': 'SVM-VGG',
                'fit_XGBoost_All_data': 'XGBoost-All Features',
                'fit_XGBoost_All_features_PCA': 'XGBoost-All Features PCA',
                'fit_XGBoost_Individual_Features_PCA': 'XGBoost-Individual Features PCA',
                'fit_XGBoost_Top3_Features': 'XGBoost-Top 3 Features',
                'fit_XGBoost_VGG': 'XGBoost-VGG',
                'Fit_Random_Forest_Classifier_All_Features_PCA' : 'Random Forest-All Features PCA',
                'Fit_Random_Forest_Classifier_Individual_Features_PCA' : 'Random Forest-Individual Features PCA',
                'Fit_Random_Forest_Classifier_All_Data' : 'Random Forest-All Features',
                'Fit_Random_Forest_Classifier_Top3_Features' : 'Random Forest-Top 3 Features',
                'Fit_Random_Forest_Classifier_VGG' : 'Random Forest-VGG',
}

In [16]:
# Read pipe-delimited file '../../Data/Compute/job_history.txt' into a DataFrame
job_history = pd.read_csv('../../Data/Compute/job_history.txt', delimiter='|')
job_history

,JobID,JobName,Partition,State,Start,End,AllocNodes,AllocCPUS,NodeList
0,385319,jupyterhub,standalone,CANCELLED by 3581,2024-03-09T14:21:23,2024-03-09T23:02:28,1,1,arwen
1,385319.batch,batch,NaN,CANCELLED,2024-03-09T14:21:23,2024-03-09T23:03:00,1,1,arwen
2,385319.extern,extern,NaN,COMPLETED,2024-03-09T14:21:23,2024-03-09T23:02:28,1,1,arwen
3,385387,transcript_ner,jsteinhardt,FAILED,2024-03-09T21:07:13,2024-03-09T21:07:19,1,1,saruman
4,385387.batch,batch,NaN,FAILED,2024-03-09T21:07:13,2024-03-09T21:07:19,1,1,saruman
...,...,...,...,...,...,...,...,...,...
856,460206.batch,batch,NaN,COMPLETED,2024-04-14T19:09:45,2024-04-14T19:30:59,1,16,balrog
857,460206.extern,extern,NaN,COMPLETED,2024-04-14T19:09:45,2024-04-14T19:30:59,1,16,balrog
858,460450,Evaluate_Autogluon_Tabular_Only_SCF_Medium_Pre...,jsteinhardt,COMPLETED,2024-04-14T20:02:09,2024-04-14T21:04:25,1,16,balrog
859,460450.batch,batch,NaN,COMPLETED,2024-04-14T20:02:09,2024-04-14T21:04:25,1,16,balrog


In [17]:
# Selected jobs - Filter job_history DataFrame to only include jobs in jobs_and_clean_names keys (job names)
# require State == 'COMPLETED'
# select row of maximum End time for each job
selected_jobs = (job_history[job_history['JobName'].isin(jobs_and_clean_names.keys())]
                                                   .query('State == "COMPLETED"')
                                                   .groupby('JobName')
                                                   .apply(lambda x: x.loc[x['End'].idxmax()])
                                                   .reset_index(drop=True)
                                                   )
# Get time elapsed by differencing Start and End times
selected_jobs['TimeElapsed'] = pd.to_datetime(selected_jobs['End']) - pd.to_datetime(selected_jobs['Start'])
selected_jobs

,JobID,JobName,Partition,State,Start,End,AllocNodes,AllocCPUS,NodeList,TimeElapsed
0,434001,Fit_Logistic_Regression_Classifier_All_Data,jsteinhardt,COMPLETED,2024-04-07T15:19:21,2024-04-08T16:23:21,1,16,shadowfax,1 days 01:04:00
1,434003,Fit_Logistic_Regression_Classifier_All_Feature...,jsteinhardt,COMPLETED,2024-04-07T15:19:38,2024-04-07T19:27:29,1,16,smokyquartz,0 days 04:07:51
2,434002,Fit_Logistic_Regression_Classifier_Individual_...,jsteinhardt,COMPLETED,2024-04-07T15:19:30,2024-04-07T22:24:49,1,16,shadowfax,0 days 07:05:19
3,460194,Fit_Logistic_Regression_Classifier_Top3_Features,epurdom,COMPLETED,2024-04-14T17:25:53,2024-04-14T20:20:36,1,16,frodo,0 days 02:54:43
4,460087,Fit_Logistic_Regression_Classifier_VGG,epurdom,COMPLETED,2024-04-14T10:23:39,2024-04-14T11:18:03,1,16,frodo,0 days 00:54:24
5,460138,Fit_Random_Forest_Classifier_All_Data,jsteinhardt,COMPLETED,2024-04-14T11:13:42,2024-04-14T14:33:54,1,16,rainbowquartz,0 days 03:20:12
6,460137,Fit_Random_Forest_Classifier_All_Features_PCA,jsteinhardt,COMPLETED,2024-04-14T11:13:28,2024-04-14T13:16:03,1,16,rainbowquartz,0 days 02:02:35
7,460136,Fit_Random_Forest_Classifier_Individual_Featur...,jsteinhardt,COMPLETED,2024-04-14T11:13:22,2024-04-14T13:06:10,1,16,rainbowquartz,0 days 01:52:48
8,460195,Fit_Random_Forest_Classifier_Top3_Features,epurdom,COMPLETED,2024-04-14T17:26:15,2024-04-14T18:14:38,1,16,frodo,0 days 00:48:23
9,460191,Fit_Random_Forest_Classifier_VGG,epurdom,COMPLETED,2024-04-14T16:44:11,2024-04-14T17:15:03,1,16,frodo,0 days 00:30:52


## CPU data

In [18]:
# Load '../../Data/Compute/scf_cpus.xlsx'
scf_cpus = pd.read_excel('../../Data/Compute/scf_cpus.xlsx')
scf_cpus

,partition,node,CPU model,passmark score
0,jsteinhardt,shadowfax,Intel Xeon Silver 4214 CPU @ 2.20GHz,16190
1,jsteinhardt,balrog,Intel Xeon Gold 5220R CPU @ 2.20GHz,33370
2,jsteinhardt,saruman,Intel Xeon Gold 5320 CPU @ 2.20GHz,37558
3,jsteinhardt,sunstone,Intel Xeon Gold 5218 CPU @ 2.30GHz,21986
4,epurdom,frodo,AMD EPYC 7543 32-Core Processor,60132
5,jsteinhardt,smokyquartz,Intel Xeon Gold 5218 CPU @ 2.30GHz,21986
6,jsteinhardt,rainbowquartz,Intel Xeon Gold 5218 CPU @ 2.30GHz,21986
7,epurdom,samwise,AMD EPYC 7543 32-Core Processor,60132


## Join on NodeList and node

In [19]:
job_cpu = pd.merge(selected_jobs, scf_cpus, left_on = 'NodeList', right_on = 'node', how = 'left')
job_cpu

,JobID,JobName,Partition,State,Start,End,AllocNodes,AllocCPUS,NodeList,TimeElapsed,partition,node,CPU model,passmark score
0,434001,Fit_Logistic_Regression_Classifier_All_Data,jsteinhardt,COMPLETED,2024-04-07T15:19:21,2024-04-08T16:23:21,1,16,shadowfax,1 days 01:04:00,jsteinhardt,shadowfax,Intel Xeon Silver 4214 CPU @ 2.20GHz,16190
1,434003,Fit_Logistic_Regression_Classifier_All_Feature...,jsteinhardt,COMPLETED,2024-04-07T15:19:38,2024-04-07T19:27:29,1,16,smokyquartz,0 days 04:07:51,jsteinhardt,smokyquartz,Intel Xeon Gold 5218 CPU @ 2.30GHz,21986
2,434002,Fit_Logistic_Regression_Classifier_Individual_...,jsteinhardt,COMPLETED,2024-04-07T15:19:30,2024-04-07T22:24:49,1,16,shadowfax,0 days 07:05:19,jsteinhardt,shadowfax,Intel Xeon Silver 4214 CPU @ 2.20GHz,16190
3,460194,Fit_Logistic_Regression_Classifier_Top3_Features,epurdom,COMPLETED,2024-04-14T17:25:53,2024-04-14T20:20:36,1,16,frodo,0 days 02:54:43,epurdom,frodo,AMD EPYC 7543 32-Core Processor,60132
4,460087,Fit_Logistic_Regression_Classifier_VGG,epurdom,COMPLETED,2024-04-14T10:23:39,2024-04-14T11:18:03,1,16,frodo,0 days 00:54:24,epurdom,frodo,AMD EPYC 7543 32-Core Processor,60132
5,460138,Fit_Random_Forest_Classifier_All_Data,jsteinhardt,COMPLETED,2024-04-14T11:13:42,2024-04-14T14:33:54,1,16,rainbowquartz,0 days 03:20:12,jsteinhardt,rainbowquartz,Intel Xeon Gold 5218 CPU @ 2.30GHz,21986
6,460137,Fit_Random_Forest_Classifier_All_Features_PCA,jsteinhardt,COMPLETED,2024-04-14T11:13:28,2024-04-14T13:16:03,1,16,rainbowquartz,0 days 02:02:35,jsteinhardt,rainbowquartz,Intel Xeon Gold 5218 CPU @ 2.30GHz,21986
7,460136,Fit_Random_Forest_Classifier_Individual_Featur...,jsteinhardt,COMPLETED,2024-04-14T11:13:22,2024-04-14T13:06:10,1,16,rainbowquartz,0 days 01:52:48,jsteinhardt,rainbowquartz,Intel Xeon Gold 5218 CPU @ 2.30GHz,21986
8,460195,Fit_Random_Forest_Classifier_Top3_Features,epurdom,COMPLETED,2024-04-14T17:26:15,2024-04-14T18:14:38,1,16,frodo,0 days 00:48:23,epurdom,frodo,AMD EPYC 7543 32-Core Processor,60132
9,460191,Fit_Random_Forest_Classifier_VGG,epurdom,COMPLETED,2024-04-14T16:44:11,2024-04-14T17:15:03,1,16,frodo,0 days 00:30:52,epurdom,frodo,AMD EPYC 7543 32-Core Processor,60132


## Compute units - compute time times num cpus times passmark score

In [20]:
# compute units - TimeElapsed * CPUs * passmark score
job_cpu['Units'] = job_cpu['TimeElapsed'].dt.total_seconds() * job_cpu['AllocCPUS'] * job_cpu['passmark score']
job_cpu

,JobID,JobName,Partition,State,Start,End,AllocNodes,AllocCPUS,NodeList,TimeElapsed,partition,node,CPU model,passmark score,Units
0,434001,Fit_Logistic_Regression_Classifier_All_Data,jsteinhardt,COMPLETED,2024-04-07T15:19:21,2024-04-08T16:23:21,1,16,shadowfax,1 days 01:04:00,jsteinhardt,shadowfax,Intel Xeon Silver 4214 CPU @ 2.20GHz,16190,2.337577e+10
1,434003,Fit_Logistic_Regression_Classifier_All_Feature...,jsteinhardt,COMPLETED,2024-04-07T15:19:38,2024-04-07T19:27:29,1,16,smokyquartz,0 days 04:07:51,jsteinhardt,smokyquartz,Intel Xeon Gold 5218 CPU @ 2.30GHz,21986,5.231261e+09
2,434002,Fit_Logistic_Regression_Classifier_Individual_...,jsteinhardt,COMPLETED,2024-04-07T15:19:30,2024-04-07T22:24:49,1,16,shadowfax,0 days 07:05:19,jsteinhardt,shadowfax,Intel Xeon Silver 4214 CPU @ 2.20GHz,16190,6.610442e+09
3,460194,Fit_Logistic_Regression_Classifier_Top3_Features,epurdom,COMPLETED,2024-04-14T17:25:53,2024-04-14T20:20:36,1,16,frodo,0 days 02:54:43,epurdom,frodo,AMD EPYC 7543 32-Core Processor,60132,1.008582e+10
4,460087,Fit_Logistic_Regression_Classifier_VGG,epurdom,COMPLETED,2024-04-14T10:23:39,2024-04-14T11:18:03,1,16,frodo,0 days 00:54:24,epurdom,frodo,AMD EPYC 7543 32-Core Processor,60132,3.140334e+09
5,460138,Fit_Random_Forest_Classifier_All_Data,jsteinhardt,COMPLETED,2024-04-14T11:13:42,2024-04-14T14:33:54,1,16,rainbowquartz,0 days 03:20:12,jsteinhardt,rainbowquartz,Intel Xeon Gold 5218 CPU @ 2.30GHz,21986,4.225533e+09
6,460137,Fit_Random_Forest_Classifier_All_Features_PCA,jsteinhardt,COMPLETED,2024-04-14T11:13:28,2024-04-14T13:16:03,1,16,rainbowquartz,0 days 02:02:35,jsteinhardt,rainbowquartz,Intel Xeon Gold 5218 CPU @ 2.30GHz,21986,2.587312e+09
7,460136,Fit_Random_Forest_Classifier_Individual_Featur...,jsteinhardt,COMPLETED,2024-04-14T11:13:22,2024-04-14T13:06:10,1,16,rainbowquartz,0 days 01:52:48,jsteinhardt,rainbowquartz,Intel Xeon Gold 5218 CPU @ 2.30GHz,21986,2.380820e+09
8,460195,Fit_Random_Forest_Classifier_Top3_Features,epurdom,COMPLETED,2024-04-14T17:26:15,2024-04-14T18:14:38,1,16,frodo,0 days 00:48:23,epurdom,frodo,AMD EPYC 7543 32-Core Processor,60132,2.793011e+09
9,460191,Fit_Random_Forest_Classifier_VGG,epurdom,COMPLETED,2024-04-14T16:44:11,2024-04-14T17:15:03,1,16,frodo,0 days 00:30:52,epurdom,frodo,AMD EPYC 7543 32-Core Processor,60132,1.781831e+09


## Get compute time, roughly, if the job was run on laptop

11th Gen Intel(R) Core(TM) i7-1165G7 @ 2.80 GHz

Passmark: 10,293

In [21]:
# Add estimated laptop time as Units divided by 10,293
job_cpu['LaptopTime'] = job_cpu['Units'] / 10293
# Convert to minutes by dividing by 60
job_cpu['LaptopTime minutes'] = job_cpu['LaptopTime'] / 60
# Convert to hours by dividing by 60
job_cpu['LaptopTime hours'] = job_cpu['LaptopTime minutes'] / 60
job_cpu

,JobID,JobName,Partition,State,Start,End,AllocNodes,AllocCPUS,NodeList,TimeElapsed,partition,node,CPU model,passmark score,Units,LaptopTime,LaptopTime minutes,LaptopTime hours
0,434001,Fit_Logistic_Regression_Classifier_All_Data,jsteinhardt,COMPLETED,2024-04-07T15:19:21,2024-04-08T16:23:21,1,16,shadowfax,1 days 01:04:00,jsteinhardt,shadowfax,Intel Xeon Silver 4214 CPU @ 2.20GHz,16190,2.337577e+10,2.271036e+06,37850.593607,630.843227
1,434003,Fit_Logistic_Regression_Classifier_All_Feature...,jsteinhardt,COMPLETED,2024-04-07T15:19:38,2024-04-07T19:27:29,1,16,smokyquartz,0 days 04:07:51,jsteinhardt,smokyquartz,Intel Xeon Gold 5218 CPU @ 2.30GHz,21986,5.231261e+09,5.082348e+05,8470.580161,141.176336
2,434002,Fit_Logistic_Regression_Classifier_Individual_...,jsteinhardt,COMPLETED,2024-04-07T15:19:30,2024-04-07T22:24:49,1,16,shadowfax,0 days 07:05:19,jsteinhardt,shadowfax,Intel Xeon Silver 4214 CPU @ 2.20GHz,16190,6.610442e+09,6.422269e+05,10703.782117,178.396369
3,460194,Fit_Logistic_Regression_Classifier_Top3_Features,epurdom,COMPLETED,2024-04-14T17:25:53,2024-04-14T20:20:36,1,16,frodo,0 days 02:54:43,epurdom,frodo,AMD EPYC 7543 32-Core Processor,60132,1.008582e+10,9.798718e+05,16331.196114,272.186602
4,460087,Fit_Logistic_Regression_Classifier_VGG,epurdom,COMPLETED,2024-04-14T10:23:39,2024-04-14T11:18:03,1,16,frodo,0 days 00:54:24,epurdom,frodo,AMD EPYC 7543 32-Core Processor,60132,3.140334e+09,3.050941e+05,5084.901661,84.748361
5,460138,Fit_Random_Forest_Classifier_All_Data,jsteinhardt,COMPLETED,2024-04-14T11:13:42,2024-04-14T14:33:54,1,16,rainbowquartz,0 days 03:20:12,jsteinhardt,rainbowquartz,Intel Xeon Gold 5218 CPU @ 2.30GHz,21986,4.225533e+09,4.105250e+05,6842.082503,114.034708
6,460137,Fit_Random_Forest_Classifier_All_Features_PCA,jsteinhardt,COMPLETED,2024-04-14T11:13:28,2024-04-14T13:16:03,1,16,rainbowquartz,0 days 02:02:35,jsteinhardt,rainbowquartz,Intel Xeon Gold 5218 CPU @ 2.30GHz,21986,2.587312e+09,2.513662e+05,4189.436964,69.823949
7,460136,Fit_Random_Forest_Classifier_Individual_Featur...,jsteinhardt,COMPLETED,2024-04-14T11:13:22,2024-04-14T13:06:10,1,16,rainbowquartz,0 days 01:52:48,jsteinhardt,rainbowquartz,Intel Xeon Gold 5218 CPU @ 2.30GHz,21986,2.380820e+09,2.313048e+05,3855.079452,64.251324
8,460195,Fit_Random_Forest_Classifier_Top3_Features,epurdom,COMPLETED,2024-04-14T17:26:15,2024-04-14T18:14:38,1,16,frodo,0 days 00:48:23,epurdom,frodo,AMD EPYC 7543 32-Core Processor,60132,2.793011e+09,2.713505e+05,4522.509045,75.375151
9,460191,Fit_Random_Forest_Classifier_VGG,epurdom,COMPLETED,2024-04-14T16:44:11,2024-04-14T17:15:03,1,16,frodo,0 days 00:30:52,epurdom,frodo,AMD EPYC 7543 32-Core Processor,60132,1.781831e+09,1.731110e+05,2885.183173,48.086386


## Load Inference Time

In [22]:
# Load get each Excel file in subdirectories of '../../Output/Classifier Inference' into a DataFrame
# and concatenate them into a single DataFrame

# Get all Excel files in subdirectories of '../../Output/Classifier Inference'
all_files = glob.glob('../../Output/Classifier Inference/**/*Prediction Statistics.xlsx', recursive=True)
all_files

# Read each Excel file into a DataFrame and concatenate them into a single DataFrame
inference_time_df = pd.concat([pd.read_excel(f) for f in all_files], ignore_index=True)
# Add column of filenames
inference_time_df['Filename Full'] = [f for f in all_files]
# Change '\\' to '/' in 'Filename Full'
inference_time_df['Filename Full'] = inference_time_df['Filename Full'].str.replace('\\', '/')
# Model - second to last part of 'Filename Full' split by '/'
inference_time_df['Model'] = inference_time_df['Filename Full'].str.split('/').str[-2]
# Features - last part of 'Filename Full' split by '/'
inference_time_df['Features'] = inference_time_df['Filename Full'].str.split('/').str[-1]
# Strip " Prediction Statistics.xlsx" from 'Features'
inference_time_df['Features'] = inference_time_df['Features'].str.replace(' Prediction Statistics.xlsx', '')
# Recode 'Features'
mapping = {'All_Features_PCA': 'All Features PCA', 'Individual_Features_PCA': 'Individual Features PCA', 'All_Data': 'All Features', 'Top3_Features': 'Top 3 Features', 'VGG': 'VGG'}
inference_time_df['Features'] = inference_time_df['Features'].replace(mapping)
# Limit columns to 'Model', 'Features', 'Inference Time (Seconds)' (runtime_minutes edited)
inference_time_df['Inference Time (Seconds)'] = inference_time_df['runtime_minutes'] * 60
inference_time_df = inference_time_df[['Model', 'Features', 'Inference Time (Seconds)']]
inference_time_df

,Model,Features,Inference Time (Seconds)
0,Logistic Regression,All Features,0.018357
1,Logistic Regression,All Features PCA,0.041061
2,Logistic Regression,Individual Features PCA,0.001303
3,Logistic Regression,Top 3 Features,0.062902
4,Logistic Regression,VGG,0.068139
5,Random Forest,All Features,0.881678
6,Random Forest,All Features PCA,0.894246
7,Random Forest,Individual Features PCA,0.345585
8,Random Forest,Top 3 Features,0.213380
9,Random Forest,VGG,0.651461


## Accuracy - Efficiency Table

In [23]:
# Load model performance table
# '../../Output/Classifier Evaluation/Model Performance Table.xlsx'
model_performance = pd.read_excel('../../Output/Classifier Evaluation/Model Performance Table.xlsx')
model_performance

,Model,Features,Tuned Hyperparameters,Train Accuracy,Mean Cross-Validated Accuracy,Test Accuracy
0,Logistic Regression,All Features,"C: 0.1, class_weight: balanced, l1_ratio: 1.0,...",0.984175,0.915708,0.910678
1,Logistic Regression,All Features PCA,"C: 0.1, class_weight: None, l1_ratio: 0.5, mul...",0.992171,0.904881,0.890531
2,Logistic Regression,Individual Features PCA,"C: 0.01, class_weight: balanced, l1_ratio: 0.2...",0.958021,0.915876,0.909335
3,Logistic Regression,Top 3 Features,"C: 0.1, class_weight: None, l1_ratio: 1.0, mul...",0.923205,0.858071,0.850235
4,Logistic Regression,VGG,"C: 0.1, class_weight: None, l1_ratio: 1.0, mul...",0.918041,0.863565,0.857623
5,SVM,All Features PCA,"C: 10.0, class_weight: balanced, gamma: 1e-05,...",0.964351,0.882725,0.898590
6,SVM,Individual Features PCA,"C: 10.0, class_weight: balanced, gamma: 1e-05,...",0.963185,0.904881,0.904634
7,SVM,Top 3 Features,"C: 10.0, class_weight: balanced, gamma: 0.0002...",0.989005,0.848908,0.838818
8,SVM,VGG,"C: 10.0, class_weight: balanced, gamma: scale,...",1.000000,0.869729,0.862324
9,XGBoost,All Features,"learning_rate: 0.5, max_depth: 3, min_child_we...",1.000000,0.908546,0.901276


In [24]:
# Cleaned up table of cpu usage
# Limit job_cpu to only include columns 'JobName', 'LaptopTime hours'
job_cpu_clean = job_cpu[['JobName', 'LaptopTime hours']]
# Create column jobname_clean as the cleaned up job name from jobs_and_clean_names
job_cpu_clean['jobname_clean'] = job_cpu_clean['JobName'].map(jobs_and_clean_names)
# Split jobname_clean into two columns 'Model' and 'Features' on '-'
job_cpu_clean[['Model', 'Features']] = job_cpu_clean['jobname_clean'].str.split('-', expand=True)
# Drop 'JobName' and 'jobname_clean' columns
job_cpu_clean.drop(columns=['JobName', 'jobname_clean'], inplace=True)
job_cpu_clean

C:\Users\ijyli\AppData\Local\Temp\ipykernel_15228\3029528637.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  job_cpu_clean['jobname_clean'] = job_cpu_clean['JobName'].map(jobs_and_clean_names)
C:\Users\ijyli\AppData\Local\Temp\ipykernel_15228\3029528637.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  job_cpu_clean[['Model', 'Features']] = job_cpu_clean['jobname_clean'].str.split('-', expand=True)
C:\Users\ijyli\AppData\Local\Temp\ipykernel_15228\3029528637.py:7: SettingWithCopyWarning: 
A value is t

,LaptopTime hours,Model,Features
0,630.843227,Logistic Regression,All Features
1,141.176336,Logistic Regression,All Features PCA
2,178.396369,Logistic Regression,Individual Features PCA
3,272.186602,Logistic Regression,Top 3 Features
4,84.748361,Logistic Regression,VGG
5,114.034708,Random Forest,All Features
6,69.823949,Random Forest,All Features PCA
7,64.251324,Random Forest,Individual Features PCA
8,75.375151,Random Forest,Top 3 Features
9,48.086386,Random Forest,VGG


In [25]:
# Accuracy - Efficiency table - join on 'Model' and 'Features'
eff_acc_table = pd.merge(model_performance, job_cpu_clean, on=['Model', 'Features'], how='left').merge(inference_time_df, on=['Model', 'Features'], how='left')
# Keep columns Model, Features, LaptopTime hours, Test Accuracy
eff_acc_table = eff_acc_table[['Model', 'Features', 'LaptopTime hours', 'Inference Time (Seconds)', 'Test Accuracy']]
# Rename LaptopTime hours to Hyperparameter Tuning + Training Time (Hours)
eff_acc_table.rename(columns={'LaptopTime hours': 'Hyperparameter Tuning + Training Time (Hours)'}, inplace=True)
eff_acc_table

,Model,Features,Hyperparameter Tuning + Training Time (Hours),Inference Time (Seconds),Test Accuracy
0,Logistic Regression,All Features,630.843227,0.018357,0.910678
1,Logistic Regression,All Features PCA,141.176336,0.041061,0.890531
2,Logistic Regression,Individual Features PCA,178.396369,0.001303,0.909335
3,Logistic Regression,Top 3 Features,272.186602,0.062902,0.850235
4,Logistic Regression,VGG,84.748361,0.068139,0.857623
5,SVM,All Features PCA,78.335112,8.747663,0.898590
6,SVM,Individual Features PCA,82.826983,14.258498,0.904634
7,SVM,Top 3 Features,76.543556,8.010652,0.838818
8,SVM,VGG,16.383645,3.604205,0.862324
9,XGBoost,All Features,174.697528,0.059770,0.901276


In [26]:
# Formatting and export
eff_acc_table = (eff_acc_table.style.format(thousands=",", decimal=".")
                                    .set_table_styles([dict(selector='th', props=[('text-align', 'center')])])
                                    .set_properties(**{'text-align': 'center'})
                                    .hide(axis="index"))

# Save to PNG
dfi.export(eff_acc_table, '../../Output/Classifier Evaluation/Efficiency-Accuracy Table.png')

# Save to Excel
eff_acc_table.to_excel('../../Output/Classifier Evaluation/Efficiency-Accuracy Table.xlsx', index=False)

eff_acc_table

Model,Features,Hyperparameter Tuning + Training Time (Hours),Inference Time (Seconds),Test Accuracy
Logistic Regression,All Features,630.843227,0.018357,0.910678
Logistic Regression,All Features PCA,141.176336,0.041061,0.890531
Logistic Regression,Individual Features PCA,178.396369,0.001303,0.909335
Logistic Regression,Top 3 Features,272.186602,0.062902,0.850235
Logistic Regression,VGG,84.748361,0.068139,0.857623
SVM,All Features PCA,78.335112,8.747663,0.898590
SVM,Individual Features PCA,82.826983,14.258498,0.904634
SVM,Top 3 Features,76.543556,8.010652,0.838818
SVM,VGG,16.383645,3.604205,0.862324
XGBoost,All Features,174.697528,0.059770,0.901276
